In [1]:
import os

In [2]:
%pwd

'/Users/anujvaghani0/Developer/projects/Chest-Cancer-Classification-using-MLflow-DVC-dagshub/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/anujvaghani0/Developer/projects/Chest-Cancer-Classification-using-MLflow-DVC-dagshub'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.chest_Cancer_Classification.constants import *
from src.chest_Cancer_Classification.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [9]:
import os
import zipfile
import gdown
from chest_Cancer_Classification import logger
from chest_Cancer_Classification.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
     
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-01-21 01:12:43,938: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-21 01:12:43,942: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-21 01:12:43,944: INFO: common: created directory at: artifacts]
[2024-01-21 01:12:43,945: INFO: common: created directory at: artifacts/data_ingestion]
[2024-01-21 01:12:43,946: INFO: 1533896105: Downloading data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY
From (redirected): https://drive.google.com/uc?/export=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY&confirm=t&uuid=7179c8ad-61ea-4e8d-97ba-93e306dee9dc
To: /Users/anujvaghani0/Developer/projects/Chest-Cancer-Classification-using-MLflow-DVC-dagshub/artifacts/data_ingestion/data.zip
100%|██████████| 49.0M/49.0M [00:28<00:00, 1.72MB/s]

[2024-01-21 01:13:14,829: INFO: 1533896105: Downloaded data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into file artifacts/data_ingestion/data.zip]
